In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))

green_taxi_df.head(10)

[Info] read from /tmp/tmp5um2rsfk/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=1/part-00175-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2745-1.c000.snappy.parquet
[Info] read from /tmp/tmpndh0jzew/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=2/part-00007-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2577-1.c000.snappy.parquet
[Info] read from /tmp/tmp5qofk96t/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=3/part-00133-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2703-1.c000.snappy.parquet
[Info] read from /tmp/tmp_85z715x/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=4/part-00073-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2643-1.c000.snappy.parquet
[Info] read from /tmp/tmp4j9u1r8o/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=5/part-00177-tid-4753095944193949832

In [ ]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour

    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

In [ ]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID",
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)

green_taxi_df.head(5)

In [ ]:
green_taxi_df.describe()

In [ ]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

In [ ]:
final_df.describe()

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws

In [ ]:
from pycaret.classification import *

automl_settings = {
    data: final_df,
    target: "totalAmount",
    log_experiment: True,
    experiment_name: "nytaxi-pycaret",
    log_plots: True,
    log_profile: True,
    silent: True
}

exp = setup(**automl_settings)